  
# Trabajo Práctico 2

## Simulación de Eventos Discretos

#### Departamento de Computación - Facultad de Ciencias Exactas y Naturales - Universidad de Buenos Aires

25 de Octubre de 2021

**Grupo 04:** Confalonieri, Gisela - Lucero, Emiliano
    

----

# Modelo conceptual

En el presente trabajo modelaremos una simplificación del funcionamiento de la toma de radiografías.

Una radiografía es una técnica diagnóstica radiológica, en la que la imagen se obtiene al exponer al receptor de imagen radiográfica a una fuente de radiación de alta energía, comúnmente rayos X o radiación gamma procedente de isótopos radiactivos. Al interponer un objeto entre la fuente de radiación y el receptor, las partes más densas aparecen con diferentes tonos dentro de una escala de grises [1]. 

Los rayos X y los rayos gamma comprenden la porción de las altas energías y cortas longitudes de onda del espectro electromagnético. Los rayos gamma y los rayos X de igual longitud de onda tienen idénticas propiedades. Características de la radiación ionizante se basan en las siguientes características de los rayos X y de los rayos gamma:

* Tienen una longitud de onda inversamente proporcional a su energía
* No tienen carga eléctrica ni tienen masa.
* En el espacio, ambos **viajan en línea recta a la velocidad de la luz.**
* **Pueden penetrar la materia; la profundidad de la penetración depende de la longitud de onda de la radiación y de la naturaleza del material que es penetrado.**
* **Son absorbidos por la materia; el porcentaje de absorción es función de la densidad y el espesor del material y de la longitud de onda de la radiación.**
* Son dispersados por la materia; la cantidad de dispersión es función de la densidad de la materia y de la longitud de onda de la radiación.
* Pueden ionizar la materia.
* Pueden exponer un film/detector por ionización.
* Pueden producir fluorescencia en ciertos materiales.
* No pueden detectarse por medio de los sentidos humanos.
[1]


<!---Los rayos X, como los rayos de la luz visible, divergen desde su fuente de emisión y cubren mayores áreas en la medida que aumenta la distancia desde la fuente. NO VAMOS A CONSIDERAR ESTO EN PRINCIPIO--->


En la medida que los fotones de alta energía pasan a través de un material, su energía decrece. Esto se conoce como **atenuación**. La teoría de atenuación dice que fotones de mayores energías viajan a través de los tejidos más fácilmente que los fotones de bajas energías (esto se relaciona mucho con el efecto fotoeléctrico)[2]. <!---La probabilidad de absorción fotoeléctrica es aproximadamente proporcional a $$(Z/E)^3$$, donde Z es el número atómico del átomo de tejido y E es la energía del fotón. A medida que E es más grande, la probabilidad de interacción cae rápidamente .--->

<!--- lo que finalmente es la "placa de radiografía" es la marca de los rayos que han logrado atravesar las partes más blandas y llegar al chasis con la placa (por eso los huesos salen blancos y los pulmones, que tienen aire, negros). 

Además, los rayos se atenúan con la distancia, por lo que cambia la cantidad de radiación necesaria según la distancia del cañón al objetivo, y la calidad de imagen que se desea obtener.--->

Entonces, en este trabajo nos interesa modelar una vista "transversal" de este proceso: simulando el disparo de fotones desde ciertos puntos en un extremo de un plano y con cierta energía inicial, su avance en el espacio en línea recta, la atenuación de su energía en la medida que recorre el aire y/o se encuentra con materiales que puedan absorber parte de la energía, y la energía con la que llegan los fotones al extremo receptor del plano, representando la imagen radiológica resultante.

Por simplicidad, no consideraremos en nuestro primer modelo los aspectos de dispersión de radiación ni divergencia desde la fuente de emisión (es decir, que los rayos cubren mayores áreas en la medida que aumenta la distancia desde la fuente). Por lo tanto, nuestro modelo sólo considerará rayos que se disparan en línea recta desde la fuente emisora hacia el extremo receptor.

![](./imgs/idea_radiografia.png)

## Estructura del modelo


* Capa de transmisión.
* Capa de obstáculos.
* Ambas capas de la misma dimensión 2D.
* Un rayo por fila, atraviesa el medio de izquierda a derecha.
* Las celdas de la capa de obtáculos reprensentan el coeficiente de atenuación del obstaculo presente en ese lugar.
* Las celdas de la capa de transmisión representan la carga de los fotones ubicado en esa posición.
* Asumimos que los fotones se mueven con un ángulo recto.
* Asumimos que no hay rebotes de fotones.
* La primera columna de la capa de transmisión contiene las cargas iniciales de los fotones.
* La última columna de la capa de transmisión contiene las cargas finales de los fotones que alcanzaron el receptor.
* Las reglas definen el movimiento de los fotones a través del tiempo y la atenuación de las cargas a medida que se encuentran con obstaculos.

# Links - Referencias
[1] [Radiography - Wikipedia](https://en.wikipedia.org/wiki/Radiography)

[2] [What is X-Ray Attenuation](https://www.radiation-dosimetry.org/what-is-x-ray-attenuation-definition/)
* [X-Ray Mass Attenuation](https://www.physics.nist.gov/PhysRefData/XrayMassCoef/chap2.html)
* [X-Ray Mass Attenuation Coefficients](https://physics.nist.gov/PhysRefData/XrayMassCoef/tab2.html)
* [Simulation of X-Ray NDT Imaging Techniques](https://www.ndt.net/article/wcndt00/papers/idn256/idn256.htm)

# Especificación

# Implementación

## General

Este modelo consta de 2 tableros, uno con las intensidades de los fotones que se mueven en el espacio y otro con las
atenuaciones de los objetos que obstaculizan a los fotones

Los tableros son de la forma n x m y tienen la misma dimension. Los bordes son nowrapped y el retado es inercial (TODO: VER SI ESTA BIEN LO DEL RETARDO).

Los valores inciales del tablero de intensidades tiene la siguiente disposición:
* Las celdas de la primer columna tiene un valor mayor a 0, se los puede cargar mediante un archivo de entrada o con un generador
* Las celdas de la ultima columna tienen de valor -1, para señalizar que representan el output del modelo.
* Las celdas de las columnas intermedia tienen de valor 0.

Los valores iniciales del tablero de atenuaciones se determinan mediante la entrada provista.

Las celdas representan espacio de X cm. TODO: Ver bien el valor.

La velocidad de propagación de los fotones entre las celdas es constante y vale X. Si es muy bajo, lo normalizamos a una magnitud razonable para el simulador y luego lo corregimos. TODO: ver bien el valor.

### Vecindario

Utilizamos un vecindario simple, donde solo nos interesa los vecinos de la izquierda y de la derecha en ambos tableros.

```
neighbors : rayos(0, -1, 0) rayos(0, 0, 0) rayos(0, 1, 0)
neighbors : rayos(0, -1, 1) rayos(0, 0, 1) rayos(0, 1, 1)
```

## Reglas
* Regla avanzar sin obstaculos
    * Si mi valor en intensidades y en atenuaciones es 0, y el valor de intensidad de mi vecino izquierdo es mayor a 0, tomo el valor de intensidad de mi vecino izquierdo
```
    rule : {(0,-1,0)} vel_propagacion {(0,0,0) = 0 and (0,0,1) = 0 and (0, -1, 0) > 0}  
```
* Regla avanzar con obstaculos
    * Si mi valor en intensidades es 0 y en atenuaciones no es 0, y el valor de intensidad de mi vecino izquierdo es mayor a 0, tomo el valor de intensidad de mi vecino izquierdo menos la atenuacion (TODO: Hacer bien el calculo de atenuacion)
```
    rule : {(0,-1,0) - (0,0,1)} vel_propagacion {(0,0,0) = 0 and (0,0,1) > 0 and (0, -1, 0) > 0}  
```
* Regla borde derecho
    * En las celdas de la última columna, se mantiene el valor -1 mientras su vecino izquierdo no tenga valor distinto de 0.
```
    rule: {(0,0,0)} 0 {cellpos(1) = m and (0, -1, 0) = 0 and (0,0,0) = -1}
```
* Regla output
    * En las celdas de la última columna, si su vecino izquierdo tiene un valor de intensidad mayor a 0, se toma ese valor.
```
    rule: {(0,-1,0)} 0 {cellpos(1) = m and (0, -1, 0) > 0}
```
* Regla default
    * Es siempre verdadera. Mantiene el valor actual de la celda.
```
   rule : {(0,0,0)} 0 { t }
```

In [1]:
from pathlib import Path

SED_HOME = Path.home().joinpath('SED')

#Directorio base donde está instalado el siumulador
CDPP_DIR = SED_HOME.joinpath('CDPP_ExtendedStates-codename-Santi')

CDPP_SRC = CDPP_DIR.joinpath('src')
CDPP_EXAMPLES = CDPP_DIR.joinpath('examples')
CDPP_SCRIPTS = CDPP_DIR.joinpath('scripts')
BASE_BIN = CDPP_SRC.joinpath('bin')
CDPP_ATOMICS = CDPP_SRC.joinpath('cd++/atomics')

CDPP_EXAMPLES_CELL = CDPP_EXAMPLES.joinpath('cell-devs')
CDPP_BIN = BASE_BIN.joinpath('cd++')
DRAWLOG_BIN = BASE_BIN.joinpath('drawlog')